In [1]:
#!pip install --upgrade oct2py
import numpy as np
import pandas as pd
from oct2py import octave
import matplotlib.pyplot as plt
import os
from scipy.optimize import root
import pickle
octave.addpath('C:/dynare/6.1/matlab') 
from IPython.display import display
#octave.addpath('/opt/homebrew/opt/dynare/lib/dynare/matlab')

    _pyeval at line 57 column 30

    _pyeval at line 57 column 30



Writes a line of code to a file that will be used in the call to dynare. 

Look through these files. All are run here to update parameters and the model. 

In [2]:
%run make_files_1.ipynb
%run par_list.m


alpha,beta,delta,sigma_a_0,rho_0,a_0,om_0,om_1,om_2,eps,a_0s,a_1s,a_2s,a_0g,a_1g,a_2g = 0.33,0.975,0.05,0.01,0.95,0,0.5,0.7,0.9,0.5,1,0.5,0.6,0.0,0.001,0.002


In [3]:
import pandas as pd

# Import each DataFrame from its respective CSV file
ir_df = pd.read_csv('ir_df.csv')
ir_df_dif = pd.read_csv('ir_df_dif.csv')
eg_df = pd.read_csv('eg_df.csv')
eg_df_raw = pd.read_csv('eg_df_raw.csv')

# Now ir_df, ir_df_dif, eg_df, and eg_df_raw contain the data from their respective CSV files

In [5]:
ir_df

,y,c,i,k,a_0
0,1.173563,0.716531,0.456993,9.019171,1.010050
1,1.172928,0.716506,0.456389,9.024607,1.009545
2,1.172330,0.716460,0.455843,9.029227,1.009066
3,1.171766,0.716395,0.455348,9.033122,1.008611
4,1.171235,0.716316,0.454901,9.036376,1.008178
...,...,...,...,...,...
95,1.162021,0.711380,0.450641,9.013831,1.000077
96,1.162017,0.711376,0.450641,9.013780,1.000073
97,1.162014,0.711373,0.450641,9.013732,1.000069
98,1.162010,0.711369,0.450641,9.013687,1.000066


We have only solved for y, c, i, and k. But we have all other items as functions of these as seen in review_2. The rest of this defines some of them and gives examples of how we might deal with the simulated data. 

In [ ]:
#y=eg.y
#a_0=eg.a_0
#c=eg.c
#i=eg.i


In [ ]:
om_2=1

In [ ]:
y=eg_df['y']


These are all things from our document. 

In [ ]:
length = len(eg.y)
#a_0=a_0s*(1+a_0g)** np.arange(length)
a_1=a_1s*(1+a_1g)** np.arange(length)
a_2=a_2s*(1+a_2g)** np.arange(length)

#a_1=a_1s*(1)** np.arange(length)
#a_2=a_2s*(1)** np.arange(length)


x_0=np.ones(length)
x_1=(om_1/om_0)**eps*(a_0/a_1)**(1-eps)
x_2=(om_2/om_0)**eps*(a_0/a_2)**(1-eps)
xb=x_0+x_1+x_2
p_0=np.ones(length)
p_1=a_0/a_1
p_2=a_0/a_2
c_0=c/xb
c_1=x_1/xb*p_0/p_1*c
c_2=x_2/xb*p_0/p_2*c
y_0=c_0+i
py_0=p_0*y_0
py_1=p_1*c_1
py_2=p_2*c_2
n_0=1-1/xb*c/y*(x_1+x_2)
n_1=x_1/xb*c/y
n_2=x_2/xb*c/y
n_0grow = (n_0 - np.roll(n_0, 1)) / np.roll(n_0, 1)
n_0grow[0] = np.nan 
n_1grow = (n_1 - np.roll(n_1, 1)) / np.roll(n_1, 1)
n_1grow[0] = np.nan 
n_2grow = (n_2 - np.roll(n_2, 1)) / np.roll(n_2, 1)
n_2grow[0] = np.nan 


y_0pr=y_0/n_0
y_1pr=c_1/n_1
y_2pr=c_2/n_2


As an example, I add some to our dataframe. 

In [ ]:
add_cols={
'y_0':y_0,
'c_0':c_0,
'c_1':c_1,   
'c_2':c_2, 
'py_0':py_0,
'py_1':py_1,   
'py_2':py_2, 
'n_0':n_0,
'n_1':n_1,   
'n_2':n_2,
'y_0pr':y_0pr,
'y_1pr':y_1pr,
'y_2pr':y_2pr,
 
}

In [ ]:
eg.df = eg.df.assign(**add_cols)

In [ ]:
fig, ([ax1, ax2,ax3],[ax4,ax5,ax6]) = plt.subplots(2, 3, figsize=(10, 6))

eg.df[['c','y', 'k', 'i']].plot(ax=ax1)
ax1.set_title('Aggregates')
ax1.set_xlabel('Index')
ax1.set_ylabel('Value')

eg.df[['py_0', 'py_1', 'py_2']].plot(ax=ax2)
ax2.set_title('Outputs')
ax2.set_xlabel('Index')
ax2.set_ylabel('Value')

eg.df[['n_0', 'n_1', 'n_2']].plot(ax=ax3)
ax3.set_title('Labor shares')
ax3.set_xlabel('Index')
ax3.set_ylabel('Value')

eg.df[['y_0pr', 'y_1pr', 'y_2pr']].plot(ax=ax4)
ax4.set_title('Productivity')
ax4.set_xlabel('Index')
ax4.set_ylabel('Value')

eg.df[['n_0', 'n_1', 'n_2']].plot(ax=ax5)
ax5.set_title('Labor shares')
ax5.set_xlabel('Index')
ax5.set_ylabel('Value')

eg.df[['n_0', 'n_1', 'n_2']].plot(ax=ax6)
ax6.set_title('Labor shares')
ax6.set_xlabel('Index')
ax6.set_ylabel('Value')




# Adjust layout and display the plot
plt.tight_layout()
plt.show()

In [ ]:
#dir(tt)

In [ ]:
check1=y-(py_0+py_1+py_2)
print(np.max(np.abs(check1)))
check2=1-(n_0+n_1+n_2)
print(np.max(np.abs(check2)))

check3=(n_1grow-n_2grow)-((1-eps)*(a_1g-a_2g))
print(np.nanmax(np.abs(check3)))

In [ ]:
df_cov=eg.df[['y','c','c_0','n_0']].cov()

In [ ]:
eg.df[['y','c','c_0']].cov()

In [ ]:

# Function to highlight the diagonal
def highlight_diagonal(data):
	attr = 'background-color: yellow'
	# Create a DataFrame with empty strings
	df_styler = pd.DataFrame('', index=data.index, columns=data.columns)
	# Set the diagonal to the desired attribute
	np.fill_diagonal(df_styler.values, attr)
	return df_styler

# Apply the function to the DataFrame
df_cov = df_cov.style.apply(highlight_diagonal, axis=None)

# Display the styled DataFrame
df_cov